## Segmentação de oleodutos subaquáticos via Detectron 2

A etapa de segmentação do oleoduto é crucial para a identificação precisa de sua posição e extensão nas imagens obtidas pelo ROV. 

Nessa etapa, o objetivo é separar o oleoduto do restante da cena, destacando-o de maneira clara e precisa. Para isso, utilizamos o framework Detectron 2, que é uma poderosa ferramenta de segmentação baseada em redes neurais convolucionais.

#### Instalação das bibliotecas necessárias para o projeto

In [ ]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities.
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.2/274.2 kB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyyaml: filename=PyYAML-5.1-cp310-cp310-linux_x86_64.whl size=44090 sha256=f01251d0ed13124e939fee6e757b6c46f756dd2d50d305d0e182cec799ae6f51
  Stored in directory: /root/.cache/pip/wheels/70/83/31/975b737609aba39a4099d471d5684141c1fdc3404f97e7f68a
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask 2022.12.1 requires pyyaml>=5.3.1, but you have pyyaml 5.1 which is incompatible.
flax 0.6.9 requires PyYAML>=5.4.1, but you have pyyaml 5.1 which is incompatibl

#### Importação das bibliotecas utilizadas

In [ ]:
import numpy as np
import pandas as pd
from skimage.draw import polygon2mask
import cv2, os, csv, random
import pycocotools

In [ ]:
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2 import config
from detectron2.config import get_cfg
from detectron2.data import *
from detectron2.structures import BoxMode
from detectron2.utils.visualizer import Visualizer

In [ ]:
actual_path = os.getcwd()
root_path = os.path.dirname(actual_path)

PATH_TO_TRAIN_MODEL = f"{root_path}/output/"

PATH_TO_TEST_OUTPUT = f"{root_path}/output/inference"

PATH_TO_LIB_FILES =  f"{root_path}/libs/"

sys.path.append(PATH_TO_LIB_FILES)

#### Leitura dos arquivos de configuração da rede e dados

In [ ]:
from importlib import reload
import test_config
from utils import *

reload(test_config)

test_config.cfg.OUTPUT_DIR = PATH_TO_TEST_OUTPUT
test_config.cfg.MODEL.WEIGHTS = os.path.join(PATH_TO_TRAIN_MODEL, "model_final.pth")

In [ ]:
database_header = ['frame_id', 'x1', 'y1', 'x2', 'y2', 'x3', 'y3', 'x4', 'y4', 'image_name']

df_database = pd.read_csv(
    '/content/drive/MyDrive/detectron2/database/database_annotations.csv',
    sep=';',
    header=None,
    names=database_header,
)

In [ ]:
from sklearn.model_selection import train_test_split

# Divide o dataframe em treino e teste (70% treino, 30% teste). Shuffle = true por padrão da
train_df, test_df = train_test_split(df_database, train_size=0.7, shuffle=True, random_state=1)

In [ ]:
test_df.shape

(12325, 10)

In [ ]:
#Aplicando schema no dataframe de test para garantir que todos os valores serão int
#Aqui eu ignoro a primeira e última coluna que são nome e path da imagem.
for column in database_header[1:-1]:
    test_df[column] = test_df[column].astype(int)

In [ ]:
print("[detectron_train.py] Creating Test DatasetCatalog...")

DatasetCatalog.register("pipeline_test_dataset", lambda: load_pipeline_dataset(test_df))
MetadataCatalog.get("pipeline_test_dataset").set(thing_classes=["pipeline"])

pipeline_train_metadata = MetadataCatalog.get("pipeline_test_dataset")

[detectron_train.py] Creating Test DatasetCatalog...


In [ ]:
from detectron2.data.datasets import convert_to_coco_json

# Crie um arquivo JSON no formato COCO para seus dados de validação
convert_to_coco_json("pipeline_test_dataset", "/content/drive/MyDrive/detectron2/runs/experimento2/config1/inference/pipeline_test_dataset_coco_format.json", False)

#### Rodando a avaliação do modelo no conjunto de teste

In [ ]:
from detectron2.modeling import build_model
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

model = build_model(test_config.cfg)
DetectionCheckpointer(model).load(os.path.join(PATH_TO_TRAIN_MODEL, "model_final.pth"))

test_loader = build_detection_test_loader(test_config.cfg, "pipeline_test_dataset")

# Crie um avaliador
evaluator = COCOEvaluator("pipeline_test_dataset", test_config.cfg, False, output_dir=PATH_TO_TEST_OUTPUT)

# Execute a avaliação
print(inference_on_dataset(model, test_loader, evaluator))

/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Loading and preparing results...
DONE (t=0.02s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=4.12s).
Accumulating evaluation results...
DONE (t=1.15s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.283
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.561
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.235
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.285
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.424
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.424
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.424
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1

#### Predict do modelo nas imagens da base teste

In [ ]:
PATH_TO_PREDICTED_IMAGES = f"{PATH_TO_TEST_OUTPUT}inference/"

def get_and_save_best_predictions(output, image_id):

  instances = output["instances"]

  # Verifique se há predições válidas
  if instances.has("pred_masks"):

      # Obtenha as máscaras binárias
      masks = instances.pred_masks.cpu().numpy()

      #Se a rede não previu instância, não tenho imagem para salvar, logo retorno.
      if not masks.any():
        return

      # Obtenha os scores das máscaras
      scores = instances.scores.cpu().numpy()

      # Encontre o índice da máscara com o maior score
      indice_mascara_max_score = np.argmax(scores)

      # Selecione apenas a máscara com o maior score
      combined_mask = masks[indice_mascara_max_score]

      # Crie uma imagem contendo somente a máscara binária
      binary_image = np.zeros_like(im)
      binary_image[combined_mask > 0] = im[combined_mask > 0]

      # Salve a imagem resultante

      cv2.imwrite(f"{PATH_TO_PREDICTED_IMAGES}/{image_id}.png", binary_image)

In [ ]:
from detectron2.modeling import build_model
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.engine import DefaultPredictor

model = build_model(test_config.cfg)
DetectionCheckpointer(model).load(os.path.join(PATH_TO_TRAIN_MODEL, "model_final.pth"))

predictor = DefaultPredictor(test_config.cfg)

# Obtenha a lista completa de imagens no dataset
dataset = DatasetCatalog.get("pipeline_test_dataset")

In [ ]:
# Itere sobre as imagens
for i, image_data in enumerate(dataset):

    im = cv2.imread(f'caminho-para-suas-imagens')

    # Faça o que for necessário com cada imagem
    output = predictor(im)
    get_and_save_best_predictions(output, image_data['image_id'])

/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
